In [3]:
from google.colab import drive
drive.mount('/content/drive')
!pip install opendatasets
!pip install pandas

Mounted at /content/drive


In [9]:
import numpy as np # linear algebra
import pandas as pd
import opendatasets as od
import pandas

od.download("https://www.kaggle.com/competitions/playground-series-s3e10")
train_df = pd.read_csv('./playground-series-s3e10/train.csv')
test_df = pd.read_csv('./playground-series-s3e10/test.csv')
target_col = train_df.columns[-1]
input_cols = train_df.columns[1:-1]
count_class_0, count_class_1 = train_df.Class.value_counts()
train_df.Class.value_counts(normalize=True).sort_index()
train_df[input_cols].nunique().sort_values(ascending=False)
from sklearn.preprocessing import MinMaxScaler

# scale numeric columns
scaler = MinMaxScaler().fit(train_df[input_cols])
train_df[input_cols] = scaler.transform(train_df[input_cols])

test_df[input_cols] = scaler.transform(test_df[input_cols])
targets = train_df[target_col]
train_inputs = train_df[input_cols]
train_targets = train_df[target_col]
test_inputs = test_df[input_cols]
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy = "minority", random_state=42)
train_inputs, train_targets = smote.fit_resample(train_inputs, train_targets)
# concatenating df3 and df4 along columns
final_df = pd.concat([train_inputs, train_targets], axis=1)
final_df.Class.value_counts(normalize=True).sort_index()
import lightgbm as lgbm
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
titanic_X_Train, titanic_X_Test, titanic_Y_Train, titanic_Y_Test = train_test_split(train_inputs, train_targets, test_size = 0.25,random_state = 42)
!pip install -U pymoo
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.factory import get_problem, get_reference_directions, get_sampling, get_crossover, get_mutation
from pymoo.operators.selection.rnd import RandomSelection
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.termination.default import DefaultMultiObjectiveTermination
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.operators.sampling.lhs import LHS
from pymoo.termination import get_termination
from pymoo.algorithms.moo.nsga2 import NSGA2

Skipping, found downloaded files in "./playground-series-s3e10" (use force=True to force download)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210079 sha256=73cfb5c995393b6b686557d1ba1d478083df3b53dc8b785d9dadb30ae9f8730c
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


In [10]:
# define the hyperparameter optimization problem
class HyperparameterOptimizationProblem(Problem):

    def __init__(self,level):
        # define the lower and upper bounds of the hyperparameters
        # n_estimators: number of trees in the forest (integer)
        # max_depth: maximum depth of each tree (integer)
        # max_features: maximum number of features (integer)
        # min_samples_leaf: minimum number of samples required to be at a leaf node (integer)
        xl = np.array([10, 2, .01])
        xu = np.array([600, 12, .9])
        self.level = level

        # initialize the problem with 4 variables and 2 objectives
        super().__init__(n_var = 3, n_obj = 2, xl = xl, xu = xu)

    def _evaluate(self, x, out, *args, **kwargs):
        # evaluate each solution (each row of x)
        f = np.zeros((x.shape[0], self.n_obj))
        for i in range(x.shape[0]):
          if self.level == 0:
            # get the hyperparameters
            n_estimators = int(x[i, 0])*10
            max_depth = int(x[i, 1])
            learning_rate = float(x[i, 2])
          else:
            n_estimators = int(x[i, 0])
            max_depth = int(x[i, 1])
            learning_rate = float(x[i, 2])



            # build and train the random forest model
            model = lgbm.LGBMClassifier(n_estimators=n_estimators,
                                           max_depth=max_depth,
                                           learning_rate = learning_rate,
                                           random_state=42,
                                           n_jobs = -1)
            model.fit(titanic_X_Train, titanic_Y_Train)

            # predict on the test set
            y_pred = model.predict(titanic_X_Test)

            # calculate the accuracy, f1 and ROC/AUC score as the objectives
            f[i, 0] = -accuracy_score(titanic_Y_Test, y_pred) # negate because we want to maximize
            f[i, 1] = -f1_score(titanic_Y_Test, y_pred) # negate because we want to maximize
            # f[i, 2] = -roc_auc_score(titanic_Y_Test, y_pred) # negate because we want to maximize


        # assign the objectives to the output dictionary
        out["F"] = f

In [ ]:
%%time
# create an instance of the problem
problem = HyperparameterOptimizationProblem(level=0)
problem1 = HyperparameterOptimizationProblem(level=1)

# create an instance of NSGA-III algorithm
algorithm = NSGA2(
    pop_size= 50,
    # ref_dirs=get_reference_directions("das-dennis", 3, n_partitions=12),
    # sampling=get_sampling("int_random"),
    # selection = RandomSelection(),
    # crossover=get_crossover("int_sbx", prob=0.9, eta=15),
    crossover = SBX(prob=1.0, prob_var=1.0),
    mutation=PolynomialMutation(prob=1.0),
    eliminate_duplicates=True)

# create an instance of NSGA-III algorithm
algorithm1 = NSGA2(
    pop_size= 25,
    # ref_dirs=get_reference_directions("das-dennis", 3, n_partitions=12),
    # sampling=get_sampling("int_random"),
    # selection = RandomSelection(),
    # crossover=get_crossover("int_sbx", prob=0.9, eta=15),
    crossover = SBX(prob=8.0, prob_var=7.0),
    mutation=PolynomialMutation(prob=6.0),
    eliminate_duplicates=True)

# create an instance of termination criterion
termination = get_termination("n_gen", 40)

# early stop
# early stop
# movement in the design space xtol
# the convergence in the constraint cv_tol
# objective space f_tol.
# maximum number of generations n_max_gen
# function evaluations n_max_evals
# termination = DefaultMultiObjectiveTermination(
#     xtol=1e-8,
#     cvtol=1e-6,
#     ftol=0.0025,
#     period=30,
#     n_max_gen=1000,
#     n_max_evals=100000
# )

# perform the optimization
res = minimize(problem,
               algorithm,
               termination,
               seed=42,
               save_history=True,
               verbose=True)
res1 = minimize(problem1,
               algorithm1,
               termination,
               seed=42,
               seed_population=res.pop,
               save_history=True,
               verbose=True)

Streaming output truncated to the last 5000 lines.
[LightGBM] [Info] Number of positive: 79842, number of negative: 80053
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012233 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 159895, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499340 -> initscore=-0.002639
[LightGBM] [Info] Start training from score -0.002639
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 79842, number of negative: 

In [15]:
# print the results
print(f"Best solution found: \nX = {res1.X.astype(int)} \nF = {-res1.F}")

Best solution found: 
X = [[599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]
 [599  11   0]] 
F = [[0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.99032752]
 [0.99028124 0.990